# Merging TRY data with iNaturalist observations

## README: 
This notebook takes the raw TRY data and combines it with the merged iNaturalist data from notebook 1. 

## Metadata

Here is the request, submitted 13 Oct 2023, downloaded 16 Oct 2023:

TRY Data Request 29376  
Only public data were requested.  
Title:  
Using iNaturalist data to characterize relationships between flower color and phenology  
Authors:  
Patrick McKenzie (Columbia University) PI  
Trait List:  
207, 3866,  
Species List:  
  
Description:  

In [1]:
import pandas as pd
import numpy as np

In [2]:
%pwd

'/Users/patrickmckenzie/googledrive/projects/flower_color_phenology/notebooks'

In [3]:
%cd ..

/Users/patrickmckenzie/googledrive/projects/flower_color_phenology


# Open raw data

In [4]:
raw_color_data = pd.read_csv(filepath_or_buffer='./raw_try_data/29376_13102023214606/29376.txt',sep='\t', encoding='ISO-8859-1')

In [5]:
raw_color_data.columns

Index(['LastName', 'FirstName', 'DatasetID', 'Dataset', 'SpeciesName',
       'AccSpeciesID', 'AccSpeciesName', 'ObservationID', 'ObsDataID',
       'TraitID', 'TraitName', 'DataID', 'DataName', 'OriglName',
       'OrigValueStr', 'OrigUnitStr', 'ValueKindName', 'OrigUncertaintyStr',
       'UncertaintyName', 'Replicates', 'StdValue', 'UnitName',
       'RelUncertaintyPercent', 'OrigObsDataID', 'ErrorRisk', 'Reference',
       'Comment', 'StdValueStr', 'Unnamed: 28'],
      dtype='object')

# The data is pretty messy. Let's try to isolate categorical colors

### find all one-word flower-color labels

In [6]:
np.unique(list(np.array([str(i) for i in raw_color_data.OrigValueStr])[np.array([len(str(i).split(' ')) for i in raw_color_data.OrigValueStr]) < 2]))

array(['/', '01/01/2015', '01/02/2015', '01/03/2015', '01/04/2015',
       '01/05/2015', '01/06/2015', '01/07/2015', '01/08/2015',
       '01/09/2015', '01/10/2015', '01/11/2015', '01/12/2015',
       '01/13/2015', '01/14/2015', '01/15/2015', '01/16/2015',
       '01/17/2015', '01/18/2015', '01/19/2015', '01/20/2015',
       '01/21/2015', '01/22/2015', '01/23/2015', '01/24/2015',
       '01/25/2015', '01/26/2015', '01/27/2015', '01/28/2015',
       '01/29/2015', '01/30/2015', '01/31/2015', '02/01/2015',
       '02/02/2015', '02/03/2015', '02/04/2015', '02/05/2015',
       '02/06/2015', '02/07/2015', '02/08/2015', '02/09/2015',
       '02/10/2015', '02/11/2015', '02/12/2015', '02/13/2015',
       '02/14/2015', '02/15/2015', '02/16/2015', '02/17/2015',
       '02/18/2015', '02/19/2015', '02/20/2015', '02/21/2015',
       '02/22/2015', '02/23/2015', '02/24/2015', '02/25/2015',
       '02/26/2015', '02/27/2015', '02/28/2015', '03/01/2015',
       '03/02/2015', '03/03/2015', '03/04/2015', '

# Try isolating real color labels
[Black,Blue,Brown,Green,Orange,Purple,Red,White,Yellow,Pink,Violet]

### Just narrow down to rows where the OrigValueStr column has one of these colors

In [7]:
simple_species_cols = raw_color_data.iloc[np.array([str(i).lower() in ["black","blue","brown","green","orange","purple","red","white","yellow","pink","violet"] for i in raw_color_data.OrigValueStr])]

### Add a column for the lowercase scientific names

In [8]:
simple_species_cols['AccSpeciesName_lower'] = [i.lower() for i in simple_species_cols['AccSpeciesName']]

/var/folders/qh/c7rwx84921n2kznn09zb2pnh0000gn/T/ipykernel_5815/2641763539.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  simple_species_cols['AccSpeciesName_lower'] = [i.lower() for i in simple_species_cols['AccSpeciesName']]


In [9]:
len(simple_species_cols.index)

17137

# Now match up to the inaturalist data

In [10]:
fullinat=pd.read_csv('./data/combined_raw_inaturalist_export.csv')

/var/folders/qh/c7rwx84921n2kznn09zb2pnh0000gn/T/ipykernel_5815/41183330.py:1: DtypeWarning: Columns (0,5,14,18,19,20,29,31) have mixed types. Specify dtype option on import or set low_memory=False.
  fullinat=pd.read_csv('./data/combined_raw_inaturalist_export.csv')


In [11]:
len(np.unique([' '.join(str(i).split(' ')[:2]) for i in fullinat.scientific_name]))

13847

## there are 13,847 total species in the inaturalist dataset

# How many of these do we have colors for?

In [12]:
names_list = []
colors_list = []
for idx, inat_obs_name in enumerate(np.unique([str(i) for i in fullinat.scientific_name])):
    subdf = simple_species_cols[simple_species_cols.AccSpeciesName_lower.eq(inat_obs_name.lower())][['AccSpeciesName_lower','OrigValueStr']]
    if len(subdf.index):
        names_list.extend(subdf['AccSpeciesName_lower'])
        colors_list.extend(subdf['OrigValueStr'])

In [13]:
len(names_list)

3385

In [14]:
len(np.unique(names_list))

2527

In [15]:
# proportion of species for which we have color data
2527 / 13847

0.18249440311980936

### We have colors for 2,527 of the species

# How many total observations have colors accompanying them?

### how many obervations are there?

In [16]:
len(fullinat.index)

1763883

### how many of these are in the color dataset?

In [17]:
color_present = 0
for spname in fullinat.scientific_name:
    color_present += ' '.join(str(spname).lower().split(' ')[:2]) in names_list # just taking the first two names -- agnostic to subspecies
color_present

673100

In [18]:
# proportion of obserations for which we have flower color
673100 / 1763883

0.381601274007403

# Make the final mapping csv
### This will include some species with repeated or different color labels

In [19]:
cleaned_matched_csv = pd.DataFrame([names_list,colors_list],index=['species_name','color']).T
cleaned_matched_csv

,species_name,color
0,abelmoschus esculentus,red
1,abelmoschus esculentus,white
2,abelmoschus esculentus,yellow
3,abrus precatorius,pink
4,abutilon fruticosum,orange
...,...,...
3380,zinnia peruviana,red
3381,zinnia peruviana,yellow
3382,zizania aquatica,Green
3383,zizaniopsis miliacea,Yellow


In [20]:
cleaned_matched_csv.to_csv('data/cleaned_matched_colors.csv')